In [1]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub

yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

2023-02-18 23:08:53.628541: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-18 23:08:57.503193: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [42]:
# scan data directories
import glob

nature_file_list = []
nature_file_list += glob.glob('../data/interim/AmbisonicSoundLibrary/nature/*')
nature_file_list += glob.glob('../data/interim/GoogleAudioSet/Outside, rural or natural/*')
nature_file_list += glob.glob('../data/interim/youtube/NatureSoundscapes/*')
nature_file_list += glob.glob('../data/interim/youtube/NomadicAmbience_nature/*')
nature_file_list += glob.glob('../data/interim/S2L_LULC/non_urban/*')
nature_file_list += glob.glob('../data/interim/S2L_LULC/urban_0_25/*')

city_file_list = []
city_file_list += glob.glob('../data/interim/GoogleAudioSet/Outside, urban or manmade/*')
city_file_list += glob.glob('../data/interim/youtube/NomadicAmbience_city/*')
city_file_list += glob.glob('../data/interim/SONYC/**/*.pkl')
city_file_list += glob.glob('../data/interim/S2L_LULC/urban_26_100/*')

nature_source_list = ['nature_'+i.rsplit('/', 3)[1]+'/'+i.rsplit('/', 3)[2] for i in nature_file_list]
city_source_list = ['city_'+i.rsplit('/', -1)[3] for i in city_file_list]

In [45]:
nature_df = pd.DataFrame({'file': nature_file_list, 'source': nature_source_list, 'category': 'nature'})
city_df = pd.DataFrame({'file': city_file_list, 'source': city_source_list, 'category': 'city'})
df_all = pd.concat([nature_df, city_df], ignore_index=True)
df_all

,file,source,category
0,../data/interim/AmbisonicSoundLibrary/nature/W...,nature_AmbisonicSoundLibrary/nature,nature
1,../data/interim/AmbisonicSoundLibrary/nature/R...,nature_AmbisonicSoundLibrary/nature,nature
2,../data/interim/AmbisonicSoundLibrary/nature/A...,nature_AmbisonicSoundLibrary/nature,nature
3,../data/interim/AmbisonicSoundLibrary/nature/W...,nature_AmbisonicSoundLibrary/nature,nature
4,../data/interim/AmbisonicSoundLibrary/nature/L...,nature_AmbisonicSoundLibrary/nature,nature
...,...,...,...
2895,../data/interim/S2L_LULC/urban_26_100/s2lam111...,city_S2L_LULC,city
2896,../data/interim/S2L_LULC/urban_26_100/s2lam006...,city_S2L_LULC,city
2897,../data/interim/S2L_LULC/urban_26_100/s2lam083...,city_S2L_LULC,city
2898,../data/interim/S2L_LULC/urban_26_100/s2lam052...,city_S2L_LULC,city


In [52]:
# randomly sampled 10% of files as validation set
test_df = df_all.groupby('source', group_keys=False).apply(lambda x: x.sample(frac=0.1, random_state=23))

In [53]:
# the remaining as the training dataset
train_df = df_all.drop(index = test_df.index)

In [54]:
# sanity check
df_all.groupby('source').count()

,file,category
source,,
city_GoogleAudioSet,170,170
city_S2L_LULC,240,240
city_SONYC,800,800
city_youtube,216,216
nature_AmbisonicSoundLibrary/nature,58,58
"nature_GoogleAudioSet/Outside, rural or natural",160,160
nature_S2L_LULC/non_urban,240,240
nature_S2L_LULC/urban_0_25,240,240
nature_youtube/NatureSoundscapes,674,674


In [55]:
test_df.groupby('source').count()

,file,category
source,,
city_GoogleAudioSet,17,17
city_S2L_LULC,24,24
city_SONYC,80,80
city_youtube,22,22
nature_AmbisonicSoundLibrary/nature,6,6
"nature_GoogleAudioSet/Outside, rural or natural",16,16
nature_S2L_LULC/non_urban,24,24
nature_S2L_LULC/urban_0_25,24,24
nature_youtube/NatureSoundscapes,67,67


In [56]:
train_df.groupby('source').count()

,file,category
source,,
city_GoogleAudioSet,153,153
city_S2L_LULC,216,216
city_SONYC,720,720
city_youtube,194,194
nature_AmbisonicSoundLibrary/nature,52,52
"nature_GoogleAudioSet/Outside, rural or natural",144,144
nature_S2L_LULC/non_urban,216,216
nature_S2L_LULC/urban_0_25,216,216
nature_youtube/NatureSoundscapes,607,607


In [58]:
train_df

,file,source,category
0,../data/interim/AmbisonicSoundLibrary/nature/W...,nature_AmbisonicSoundLibrary/nature,nature
1,../data/interim/AmbisonicSoundLibrary/nature/R...,nature_AmbisonicSoundLibrary/nature,nature
2,../data/interim/AmbisonicSoundLibrary/nature/A...,nature_AmbisonicSoundLibrary/nature,nature
3,../data/interim/AmbisonicSoundLibrary/nature/W...,nature_AmbisonicSoundLibrary/nature,nature
4,../data/interim/AmbisonicSoundLibrary/nature/L...,nature_AmbisonicSoundLibrary/nature,nature
...,...,...,...
2894,../data/interim/S2L_LULC/urban_26_100/s2lam091...,city_S2L_LULC,city
2895,../data/interim/S2L_LULC/urban_26_100/s2lam111...,city_S2L_LULC,city
2896,../data/interim/S2L_LULC/urban_26_100/s2lam006...,city_S2L_LULC,city
2898,../data/interim/S2L_LULC/urban_26_100/s2lam052...,city_S2L_LULC,city


# Customize model

In [20]:
my_classes = ['city', 'nature']
my_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1024), dtype=tf.float32,
                          name='input_embedding'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(len(my_classes))
], name='my_model')

my_model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 525,826
Trainable params: 525,826
Non-trainable params: 0
_________________________________________________________________
